In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
import numpy as np
path = '/home/adarsh-binny/HULK/Saved_models/'

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("../HULK/Counterspeech/models/createdebate_model",cache_dir=path)
model = AutoModelWithLMHead.from_pretrained("../HULK/Counterspeech/models/createdebate_model",cache_dir=path)

In [ ]:
model.eval()

In [ ]:
conditioned_tokens = []
generated_tokens = []
past = None
fst = True
DELIMITER = "/n"

def reinput(user_msg):
    global conditioned_tokens; global fst
    os.system('cls' if os.name == 'nt' else 'clear')

    conditioned_tokens = tokenizer.decode(conditioned_tokens)

    print("Me: " + user_msg + "\n" + "Bot: ",end='')

    if fst:
      fst = False
      user_msg = "<|endoftext|>" + user_msg

    user_msg = "" + user_msg
    conditioned_tokens += user_msg 
    conditioned_tokens = tokenizer.encode(conditioned_tokens) 
    conditioned_tokens += [50256] # Append operator to prepend conversation history



def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
  """
  Credit: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  assert logits.dim() == 1  # batch size 1 for single word generation
  sorted_logits, sorted_indices = torch.sort(logits, descending=True)
  cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
  # remove tokens with cumulative probability above the threshold
  sorted_indices_to_remove = cumulative_probs > top_p
  # shift the indices to the right to keep also the first token above the threshold
  sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
  sorted_indices_to_remove[..., 0] = 0
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[indices_to_remove] = filter_value
  return logits


def recalc(p=None):
    global conditioned_tokens
    global generated_tokens
    global past

    # for segment display purpose, keep 2 sets of tokens
    indexed_tokens = conditioned_tokens + generated_tokens

    tokens_tensor = torch.tensor([indexed_tokens])

    #tokens_tensor = tokens_tensor
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    logits = predictions[0, -1, :]
    filtered_logits = top_p_filtering(logits)
    probabilities = F.softmax(filtered_logits, dim=-1)
    next_token = torch.multinomial(probabilities, 1)
    generated_tokens.append(next_token.item())
    return next_token.item()

def generate():
    global conditioned_tokens
    global generated_tokens
    global past

    while True:
        if len(tokenizer.decode(conditioned_tokens)) > 320:
          dc = tokenizer.decode(conditioned_tokens)
          dc = dc[len(dc)-320:]
          idx = dc.find(DELIMITER)
          if idx != -1:
            dc = dc[idx+len(DELIMITER):]
            conditioned_tokens = tokenizer.encode(dc)

        result = recalc()

        if result == 50256:

      # end-of-text : 50256
      # use this special token to split segments

            decoded_reply = tokenizer.decode(generated_tokens)

            to_print = decoded_reply
            if to_print.endswith("<|endoftext|>"):
              to_print = to_print[:-len("<|endoftext|>")]
              decoded_reply = to_print
            print(to_print)

            decoded_reply = "" + decoded_reply
            decoded_reply = decoded_reply+DELIMITER
            conditioned_tokens += (tokenizer.encode(decoded_reply))

            # Remove end of text tokens from feeding back into model
      # -- see here for reason https://github.com/huggingface/transformers/issues/429#issuecomment-479380117
            cond_str = tokenizer.decode(conditioned_tokens)
            cond_str = cond_str.replace("<|endoftext|>",DELIMITER) # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
            conditioned_tokens = tokenizer.encode(cond_str)

    # Uncomment to debug (print) conversation history that is re-fed to the model
    # cond_str = tokenizer.decode(conditioned_tokens)
    # cond_str = cond_str.replace("<|endoftext_|>","\n").replace(DELIMITER,"\n") # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
    # print("(condstr_dbg) " + cond_str)
    # print(r"(/constr_dbg)")

            generated_tokens = []
            break

In [ ]:
from IPython.display import clear_output
import logging

from IPython.display import clear_output
import logging

def new_dialogue():

  global conditioned_tokens
  global generated_tokens
  global past; global fst
  
  conditioned_tokens = []; generated_tokens = []; past = None; fst = True
  reinput("What is the meaning of life?")
  generate()

  while True:
    cmd = input()

    if cmd == "reset":
      clear_output()
      new_dialogue()
      break

    if cmd != "":
      reinput(cmd)
    generate()
    

In [ ]:
new_dialogue()

In [2]:
import neptune.new as neptune
from apiconfig import project_name,api_token

run = neptune.init(project=project_name,api_token=api_token)

# Track metadata and hyperparameters of your run
run["JIRA"] = "NPT-952"
run["parameters"] = {"learning_rate": 0.001,
                     "optimizer": "Adam"}

# Track the training process by logging your training metrics
for epoch in range(100):
   run["train/accuracy"].log(epoch * 0.6)
   run["train/loss"].log(epoch * 0.4)

run["f1_score"] = 0.66
run.stop()

https://app.neptune.ai/Hatespeech-CNERG/Counterspeech-generation/e/COUNTER-2


In [ ]:
!pip install neptune-client==0.9